In [1]:
!python -m pip install --upgrade pip
!pip install ipykernel
!pip install pyyaml==6.0.1
!pip install boto3==1.34.58
!pip install sdv==1.10.0
!pip install smartnoise-synth==1.0.3 # Error can be ignored
!pip install anonymeter==1.0.0
!pip install git+https://github.com/ryan112358/private-pgm.git
!pip install --upgrade torch==2.2.1 # Error can be ignored
!pip install requests==2.31.0

  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
  Using cached boto3-1.34.58-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.34.70-py3-none-any.whl.metadata (5.7 kB)
Using cached boto3-1.34.58-py3-none-any.whl (139 kB)
Using cached botocore-1.34.70-py3-none-any.whl (12.0 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.55
    Uninstalling botocore-1.34.55:
      Successfully uninstalled botocore-1.34.55
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.55
    Uninstalling boto3-1.34.55:
      Successfully uninstalled boto3-1.34.55
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.55

In [2]:
import pandas as pd
import numpy as np

In [3]:
# %cd /Users/alex/PETsARD

import os
import sys

path_petsard = os.path.dirname(os.path.dirname(os.getcwd()))
print(path_petsard)
sys.path.append(path_petsard)

/home/ec2-user/SageMaker/PETsARD


In [6]:
%cd /home/ec2-user/SageMaker/PETsARD

/home/ec2-user/SageMaker/PETsARD


In [8]:
from PETsARD import (
    Loader,
    Processor,
    Synthesizer
)


load = Loader(
    filepath='benchmark://adult-income',
    na_values={k: '?' for k in [
        'workclass',
        'occupation',
        'native-country'
    ]}
)
load.load()

Loader - Benchmarker : Success download the benchmark dataset from https://petsard-benchmark.s3.amazonaws.com/adult-income.csv.


In [9]:
load.data = load.data.loc[:1000, :]

In [10]:
import pprint


def issue332(
        load: Loader,
        synthesizing_method: str,
        scaler_inhibit: bool = False
):
    pp = pprint.PrettyPrinter(depth=2)

    proc_discretizing = Processor(
        metadata=load.metadata,
    )

    if scaler_inhibit:
        print("Preproc config of Scaler before update as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])
        proc_discretizing.update_config(
            {'scaler': {
                col: None for col in load.data.columns
            }}
        )
        print("Preproc config of Scaler before after as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])

    proc_discretizing.fit(
        data=load.data,
        sequence=[
            'missing',
            'outlier',
            'scaler',
            'discretizing'
        ]
    )
    preproc_discretizing_data = proc_discretizing.transform(
        data=load.data
    )
    print("Preproc data as ...")
    print(preproc_discretizing_data.head(1))


    syn = Synthesizer(
        method=synthesizing_method,
        epsilon=10.0,
    )
    syn.create(data=preproc_discretizing_data)
    syn.fit_sample()
    print("Sync data as ...")
    print(syn.data_syn.head(1))
    postproc_discretizing_data = proc_discretizing.inverse_transform(
        data=syn.data_syn
    )
    print(postproc_discretizing_data.head(1))

In [13]:
import pprint


def issue332_gan(
        load: Loader,
        synthesizing_method: str,
        scaler_inhibit: bool = False,
        outlier_inhibit: bool = False
):
    pp = pprint.PrettyPrinter(depth=2)

    proc_discretizing = Processor(
        metadata=load.metadata,
    )

    if scaler_inhibit:
        print("Preproc config of Scaler before update as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])
        proc_discretizing.update_config(
            {'scaler': {
                col: None for col in load.data.columns
            }}
        )
        print("Preproc config of Scaler before after as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])

    if outlier_inhibit:
        print("Preproc config of Outlier before update as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])
        proc_discretizing.update_config(
            {'outlier': {
                col: None for col in load.data.columns
            }}
        )
        print("Preproc config of Outlier before after as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])


    proc_discretizing.fit(
        data=load.data,
        sequence=[
            'missing',
            'outlier',
            'encoder',
            'scaler'
        ]
    )
    preproc_discretizing_data = proc_discretizing.transform(
        data=load.data
    )
    print("Preproc data as ...")
    print(preproc_discretizing_data.head(1))


    syn = Synthesizer(
        method=synthesizing_method,
        epsilon=10.0,
    )
    syn.create(data=preproc_discretizing_data)
    syn.fit_sample()
    print("Sync data as ...")
    print(syn.data_syn.head(1))
    postproc_discretizing_data = proc_discretizing.inverse_transform(
        data=syn.data_syn
    )
    print(postproc_discretizing_data.head(1))
    
    return postproc_discretizing_data

In [ ]:
issue332(
    load=load,
    synthesizing_method='smartnoise-aim'
)

上述執行會出現 `RuntimeError: all elements of input should be between 0 and 1`，原因是 input 中含有 NA 值，來自於前處理過程中將離群值去除掉，導致 `capital-gain`, `capital-loss` 兩個欄位的值皆相同，在 `smartnoise` 中進行 `MinMaxTransformer` 時會讓運算結果為 NA （分母為 0）。因此針對 adult 資料集，將其中的 `outlier` 前處理拿掉，就可以正常運行了。

In [ ]:
issue332(
    load=load,
    synthesizing_method='smartnoise-mst'
)

In [ ]:
issue332(
    load=load,
    synthesizing_method='smartnoise-pacsynth'
)

In [14]:
a = issue332_gan(
        load = load,
        synthesizing_method = 'smartnoise-dpctgan'
)

Preproc data as ...
        age  workclass    fnlwgt  education  educational-num  marital-status  \
0 -0.953119   0.412076  0.409053   0.794901        -1.152735        0.584823   

   occupation  relationship      race    gender  capital-gain  capital-loss  \
0    0.744022      0.753279  0.909944  0.482492     -0.159918     -0.219043   

   hours-per-week  native-country    income  
0       -0.044504        0.568916  0.365424  
Synthesizer (SmartNoise): Fitting dpctgan.
Epoch 1, Loss G: 0.7441, Loss D: 1.3932
epsilon is 0.07018358957582388, alpha is 63.0
Epoch 2, Loss G: 0.7408, Loss D: 1.3951
epsilon is 0.9199255498941705, alpha is 15.0


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/opacus/privacy_engine.py:638: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/opacus/privacy_engine.py:229: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1352: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-fu

Epoch 3, Loss G: 0.7372, Loss D: 1.3989
epsilon is 1.345212903926586, alpha is 10.9
Epoch 4, Loss G: 0.7329, Loss D: 1.3964
epsilon is 1.682835245192693, alpha is 9.4
Epoch 5, Loss G: 0.7296, Loss D: 1.4041
epsilon is 1.9753140629438175, alpha is 8.4
Epoch 6, Loss G: 0.7205, Loss D: 1.3978
epsilon is 2.23841971541107, alpha is 7.6
Epoch 7, Loss G: 0.7203, Loss D: 1.4060
epsilon is 2.4803793213336522, alpha is 7.1
Epoch 8, Loss G: 0.7150, Loss D: 1.4065
epsilon is 2.706371766594642, alpha is 6.7
Epoch 9, Loss G: 0.7151, Loss D: 1.4027
epsilon is 2.9196946333045988, alpha is 6.3
Epoch 10, Loss G: 0.7109, Loss D: 1.4057
epsilon is 3.1224734529915024, alpha is 6.1
Epoch 11, Loss G: 0.7058, Loss D: 1.4039
epsilon is 3.3163425520480274, alpha is 5.8
Epoch 12, Loss G: 0.7037, Loss D: 1.4083
epsilon is 3.5027702769562024, alpha is 5.6
Epoch 13, Loss G: 0.7028, Loss D: 1.4051
epsilon is 3.6827100008205824, alpha is 5.4
Epoch 14, Loss G: 0.6993, Loss D: 1.4077
epsilon is 3.8570107337725466, alph

In [16]:
a.nunique()

age                292
workclass            6
fnlwgt             558
education           11
educational-num    558
marital-status       5
occupation           8
relationship         6
race                 3
gender               2
capital-gain         1
capital-loss         1
hours-per-week     557
native-country       7
income               2
dtype: int64

In [ ]:
issue332_gan(
        load = load,
        synthesizing_method = 'smartnoise-patectgan',
        outlier_inhibit = True
)